<a href="https://colab.research.google.com/github/sofiaaic/PrediccionTendenciasPoliticasChile/blob/joaquin/notebooks/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Acá dejaremos el codigo para extraer los datos RAW.

In [3]:
# (Opcional) instala motores para Excel si hace falta
!pip -q install openpyxl


In [4]:
# 1) Montar Drive
from google.colab import drive

CARPETA = '/content/drive/MyDrive/data_votantes_chile'  # ajusta a tu ruta
drive.mount('/content/drive')  # autoriza con tu cuenta

Mounted at /content/drive


In [ ]:

# 2) Configurar ruta de carpeta en tu Drive
#    Ejemplo: "Mi unidad/Proyectos/Excels"
import os, glob
import pandas as pd

CARPETA = '/content/drive/MyDrive/data_votantes_chile'  # ajusta a tu ruta


# 3) Buscar archivos .xlsx (evitar temporales ~$.xlsx)
rutas = [p for p in glob.glob(os.path.join(CARPETA, '**', '*.xlsx'), recursive=True)
         if not os.path.basename(p).startswith('~$')]
# ===============================================================
# 4) Leer y concatenar las hojas "Votación en Chile"
# ===============================================================
dfs = []

for ruta in rutas:
    nombre_archivo = os.path.basename(ruta)
    print(f"\n🔍 Leyendo archivo: {nombre_archivo}")

    try:
        df = pd.read_excel(
            ruta,
            sheet_name='Votación en Chile',
            skiprows=6,
            engine='openpyxl'
        )
        df['__archivo_origen'] = nombre_archivo
        dfs.append(df)
        print(f"✅ Leído correctamente: {nombre_archivo} ({df.shape[0]} filas)")

    except ValueError as e:
        # Si la hoja no existe, mostramos las disponibles
        print(f"⚠️ No se encontró la hoja 'Votación en Chile' en {nombre_archivo}")
        try:
            xls = pd.ExcelFile(ruta, engine='openpyxl')
            print("   🗒️ Hojas disponibles:", xls.sheet_names)
        except Exception as e2:
            print(f"   ❌ No se pudieron listar las hojas: {e2}")
    except Exception as e:
        print(f"❌ Error general leyendo {nombre_archivo}: {e}")

# ===============================================================
# 5) Concatenar todos los DataFrames válidos
# ===============================================================
if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
    print(f"\n✅ Total concatenado: {df_total.shape[0]} filas, {df_total.shape[1]} columnas")
else:
    print("\n⚠️ No se pudieron leer hojas válidas.")
    df_total = pd.DataFrame()

# ===============================================================
# 6) Guardar DataFrame final como Pickle (solo si no existe)
# ===============================================================
import os

if not df_total.empty:
    ruta_salida = os.path.join(CARPETA, 'df_votacion_chile.pkl')

    if not os.path.exists(ruta_salida):
        df_total.to_pickle(ruta_salida)
        print(f"💾 DataFrame guardado en: {ruta_salida}")
    else:
        print(f"⚠️ El archivo Pickle ya existe en: {ruta_salida}")
        print("   No se guardó para evitar sobrescribirlo.")
else:
    print("⚠️ No se guardó el Pickle (DataFrame vacío).")



🔍 Leyendo archivo: 2021_12_Presidencial_2V_Datos_Eleccion.xlsx
✅ Leído correctamente: 2021_12_Presidencial_2V_Datos_Eleccion.xlsx (186556 filas)

🔍 Leyendo archivo: 2021_11_Presidencial_1V_Datos_Eleccion.xlsx
✅ Leído correctamente: 2021_11_Presidencial_1V_Datos_Eleccion.xlsx (419751 filas)

🔍 Leyendo archivo: 2021_07_PresidencialPrimarias_Datos_Eleccion.xlsx
